In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 903.1/903.1 kB 16.3 MB/s eta 0:00:00





***Import Necessary Libraries***

In [ ]:
import os
import cv2
import yaml
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO

***Load and Explore the Dataset***

In [ ]:
def load_yaml(dataset_path: str, yaml_filename: str) -> dict:
    """Loads data from a YAML file.

    Args:
      dataset_path: The path to the dataset directory.
      yaml_filename: The name of the YAML file.

    Returns:
      A dictionary containing the loaded YAML data.
    """
    yaml_file_path = os.path.join(dataset_path, yaml_filename)

    try:
        with open(yaml_file_path, 'r') as file:
            yaml_content = yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"YAML file not found: {yaml_file_path}")
    except yaml.YAMLError as e:
        raise ValueError(f"Error parsing YAML file: {e}")

    return yaml_content

# Define the dataset path and YAML filename
dataset_path = '/aug-asl-dataset'
yaml_filename = 'data.yaml'

# Load the YAML file
yaml_content = load_yaml(dataset_path, yaml_filename)
print(yaml.dump(yaml_content, default_flow_style=False))

***Analyze the Images***

In [ ]:
def analyze_images(dataset_path: str, data_type: str) -> tuple:
    """Analyzes image sizes and counts in a given dataset directory.

    Args:
      dataset_path: The path to the dataset directory.
      data_type: The type of data ('train' or 'valid').

    Returns:
      A tuple containing the number of images and a set of unique image sizes.
    """

    images_path = os.path.join(dataset_path, data_type, 'images')
    num_images = 0
    image_sizes = set()

    try:
        for filename in os.listdir(images_path):
            if filename.endswith('.jpg'):
                num_images += 1
                image_path = os.path.join(images_path, filename)
                with Image.open(image_path) as img:
                    image_sizes.add(img.size)
    except FileNotFoundError:
        raise FileNotFoundError(f"Image directory not found: {images_path}")

    return num_images, image_sizes

# Analyze the training images
num_train_images, train_image_sizes = analyze_images(dataset_path, 'train')
print(f"Number of training images: {num_train_images}")
if len(train_image_sizes) == 1:
    print(f"All training images have the same size: {train_image_sizes.pop()}")
else:
    print("Training images have varying sizes.")

# Analyze the validation images
num_valid_images, valid_image_sizes = analyze_images(dataset_path, 'valid')
print(f"Number of validation images: {num_valid_images}")
if len(valid_image_sizes) == 1:
    print(f"All validation images have the same size: {valid_image_sizes.pop()}")
else:
    print("Validation images have varying sizes.")

***Visualize Sample Images***

In [ ]:
def display_sample_images(images_path: str, num_samples: int = 8, grid_shape: tuple = (2, 4)) -> None:
    """Displays a grid of sample images from a directory.

    Args:
      images_path: The path to the directory containing images.
      num_samples: The number of sample images to display.
      grid_shape: The shape of the grid to display the images in (rows, columns).
    """

    try:
        image_files = [file for file in os.listdir(images_path) if file.endswith('.jpg')]

        if len(image_files) < num_samples:
            raise ValueError(f"Not enough images in the directory to display {num_samples} samples.")

        # Select images at equal intervals
        num_images = len(image_files)
        selected_images = [image_files[i] for i in range(0, num_images, num_images // num_samples)]

        # Create a subplot
        fig, axes = plt.subplots(*grid_shape, figsize=(20, 11))

        # Display each of the selected images
        for ax, img_file in zip(axes.ravel(), selected_images):
            img_path = os.path.join(images_path, img_file)
            image = Image.open(img_path)
            ax.imshow(image)
            ax.axis('off')

        plt.suptitle('Sample Images from Training Dataset', fontsize=20)
        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        raise FileNotFoundError(f"Image directory not found: {images_path}")

# Display sample images from the training set
train_images_path = os.path.join(dataset_path, 'train', 'images')
display_sample_images(train_images_path)

***Train the YOLOv8 Model***

In [ ]:
def train_yolo_model(
    yaml_file_path: str,
    epochs: int = 20,
    imgsz: int = 640,
    device: int = 0,
    patience: int = 50,
    batch: int = 32,
    optimizer: str = 'auto',
    lr0: float = 0.0001,
    lrf: float = 0.1,
    dropout: float = 0.1,
    seed: int = 0
) -> YOLO:
    """Trains a YOLO model on a custom dataset.

    Args:
      yaml_file_path: Path to the dataset configuration file (data.yaml).
      epochs: Number of epochs to train for.
      imgsz: Size of input images as integer.
      device: Device to run on (i.e., cuda device=0).
      patience: Epochs to wait for no observable improvement for early stopping.
      batch: Number of images per batch.
      optimizer: Optimizer to use (choices=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]).
      lr0: Initial learning rate.
      lrf: Final learning rate (lr0 * lrf).
      dropout: Use dropout regularization.
      seed: Random seed for reproducibility.

    Returns:
        The trained YOLO model.
    """

    try:
        model = YOLO('yolov8m.pt')  # Load a pretrained YOLO model
        results = model.train(
            data=yaml_file_path,
            epochs=epochs,
            imgsz=imgsz,
            device=device,
            patience=patience,
            batch=batch,
            optimizer=optimizer,
            lr0=lr0,
            lrf=lrf,
            dropout=dropout,
            seed=seed
        )
        return model

    except Exception as e:
        print(f"An error occurred during training: {e}")
        return None

# Train the model
yaml_file_path = os.path.join(dataset_path, 'data.yaml')
trained_model = train_yolo_model(yaml_file_path)

***Validate the Model and Display Inferences***

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def display_training_results(folder_path):
  """
  Displays the training result images (plots and visualizations)
  from a specified folder.

  Args:
    folder_path (str): The path to the folder containing the result images.
  """

  # Get a list of all files in the folder
  all_files = os.listdir(folder_path)

  # Filter for image files (you can add more extensions if needed)
  image_files = [
      f for f in all_files
      if f.endswith(('.png', '.jpg', '.jpeg'))
  ]

  # Calculate the number of rows and columns for subplots
  num_images = len(image_files)
  num_cols = 4  # You can adjust this
  num_rows = (num_images + num_cols - 1) // num_cols

  # Create a figure and axes for the subplots
  fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 20))
  fig.suptitle('YOLOv8 Training Results', fontsize=16)

  # Flatten the axes array for easier iteration
  axes = axes.flatten()

  for i, img_name in enumerate(image_files):
    img_path = os.path.join(folder_path, img_name)

    try:
      # Load the image
      img = mpimg.imread(img_path)

      # Display the image on the corresponding subplot
      axes[i].imshow(img)
      axes[i].set_title(img_name.split('.')[0])
      axes[i].axis('off')

    except FileNotFoundError:
      print(f"Image not found: {img_path}")
      axes[i].axis('off')

  # Hide any empty subplots
  for j in range(i + 1, len(axes)):
    axes[j].axis('off')

  # Adjust layout for better spacing
  plt.tight_layout()
  plt.show()

# Example usage:
folder_path = 'current-dir/runs/detect/train'
display_training_results(folder_path)

In [ ]:
def display_validation_inferences(model: YOLO, dataset_path: str, num_samples: int = 9) -> None:
    """Displays inferences on sample images from the validation set.

    Args:
        model: The trained YOLO model.
        dataset_path: The path to the dataset directory.
        num_samples: The number of sample images to display.
    """

    if model is None:
        print("Model not trained. Skipping validation inference.")
        return

    valid_images_path = os.path.join(dataset_path, 'valid', 'images')

    try:
        # List all jpg images in the directory
        image_files = [file for file in os.listdir(valid_images_path) if file.endswith('.jpg')]

        if len(image_files) < num_samples:
            raise ValueError(f"Not enough images in the directory to display {num_samples} samples.")

        # Select images at equal intervals
        num_images = len(image_files)
        selected_images = [image_files[i] for i in range(0, num_images, num_images // num_samples)]

        # Initialize the subplot
        fig, axes = plt.subplots(3, 3, figsize=(20, 21))
        fig.suptitle('Validation Set Inferences', fontsize=24)

        # Perform inference on each selected image and display it
        for i, ax in enumerate(axes.flatten()):
            image_path = os.path.join(valid_images_path, selected_images[i])
            results = model.predict(source=image_path, imgsz=640, conf=0.5)
            annotated_image = results[0].plot(line_width=1)
            annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
            ax.imshow(annotated_image_rgb)
            ax.axis('off')

        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        raise FileNotFoundError(f"Image directory not found: {valid_images_path}")

# Display inferences on validation set
display_validation_inferences(trained_model, dataset_path)

In [ ]:
def display_sample_inference(model: YOLO, sample_image_path: str) -> None:
    """Displays the inference on a single sample image.

    Args:
        model: The trained YOLO model.
        sample_image_path: The path to the sample image.
    """

    if model is None:
        print("Model not trained. Skipping sample inference.")
        return

    try:
        # Perform inference
        results = model.predict(source=sample_image_path, imgsz=640, conf=0.7)

        # Annotate and convert image to numpy array
        sample_image = results[0].plot(line_width=2)

        # Convert the color of the image from BGR to RGB
        sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

        # Display annotated image
        plt.figure(figsize=(20,15))
        plt.imshow(sample_image)
        plt.title('Detected Objects in Sample Image by the Fine-tuned YOLOv8 Model', fontsize=20)
        plt.axis('off')
        plt.show()

    except Exception as e:
        print(f"An error occurred during sample inference: {e}")


# Path to a sample image file for inference
sample_image_path = '/asl-dataset/test/images/I17.jpg'
display_sample_inference(trained_model, sample_image_path)